In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
table = "ETHUSDT_1m_complete_history_long_short"

In [4]:
df_long = pd.read_sql_query(f'select * from {table}', connection)

In [5]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    #df_backtesting["time"] = df_long["time"]
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    #df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [6]:
# df_backtesting = df_backtesting[:1000000]
# df_long = df_long[:1000000]

In [7]:
# def calculate_osc(data):
    
   
#     df_temp = pd.DataFrame()    
#     stock = StockDataFrame.retype(data)
    
#     df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
#     return df_temp["stochastic_oscillator"]

# def calucalte_rsi(data):
    
#     df_temp = pd.DataFrame()    
#     stock = StockDataFrame.retype(data)
    
#     df_temp["relative_strength_index"] = stock.get("rsi_30")
    
#     return df_temp["relative_strength_index"]

# def momentum(df, lag):
#     return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['close_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

def init_short_signal(df):
    
    return df_long['short_indicator']

def init_close_short_signal(df):
    
    return df_long['short_close_indicator']
    

In [8]:
class OscRsi_5m_long_short(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        #self.osc = self.I(calculate_osc, self.data.df)
        #self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        #self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        #self.close_short_init = self.I(init_close_short_signal, self.data.df)
        #self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        #self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        # elif self.close_buy_init[-1] == 1 and self.position.is_long:
        #     self.position.close()
             
             
        elif self.short_init[-1] == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()
            
        # elif self.close_short_init[-1] == 1 and self.position.is_short:
        #      self.position.close()


In [9]:
bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)
stats = bt.run()
stats

<ipython-input-9-8d03f2706277>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)


Start                                       0
End                               2.74946e+06
Duration                          2.74946e+06
Exposure Time [%]                     99.6333
Equity Final [$]                  1.00714e+07
Equity Peak [$]                   1.71785e+07
Return [%]                             100614
Buy & Hold Return [%]                 29047.7
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                    -73.9174
Avg. Drawdown [%]                    -1.84287
Max. Drawdown Duration                 323467
Avg. Drawdown Duration                1671.69
# Trades                                  270
Win Rate [%]                          42.5926
Best Trade [%]                        436.049
Worst Trade [%]                      -19.8608
Avg. Trade [%]                    

In [23]:
stats["_equity_curve"]

,Equity,DrawdownPct,DrawdownDuration
0,1.000000e+04,0.000000,NaN
1,1.000000e+04,0.000000,NaN
2,1.000000e+04,0.000000,NaN
3,1.000000e+04,0.000000,NaN
4,1.000000e+04,0.000000,NaN
...,...,...,...
2749458,1.004510e+07,0.415251,NaN
2749459,1.005839e+07,0.414477,NaN
2749460,1.006762e+07,0.413940,NaN
2749461,1.007147e+07,0.413716,NaN


In [11]:
#bt.plot()

In [12]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [13]:
stats._strategy

<Strategy OscRsi_5m_long_short>

In [14]:
stats['_trades']


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1093,10082,17301,9.144846,9.950,-8.800333e+02,-0.088045,10082,17301,7219
1,915,17301,31945,9.959950,13.733,3.452341e+03,0.378822,17301,31945,14644
2,-916,31945,32438,13.719267,13.940,-2.021914e+02,-0.016089,31945,32438,493
3,886,32438,42010,13.953940,12.659,-1.147317e+03,-0.092801,32438,42010,9572
4,-887,42010,45728,12.646341,14.890,-1.990126e+03,-0.177416,42010,45728,3718
...,...,...,...,...,...,...,...,...,...,...
265,2522,2685604,2688504,3117.754640,2973.600,-3.635580e+05,-0.046237,2685604,2688504,2900
266,-2525,2688504,2701299,2970.626400,2775.100,4.937042e+05,0.065820,2688504,2701299,12795
267,2878,2701299,2710628,2777.875100,2603.480,-5.019091e+05,-0.062780,2701299,2710628,9329
268,-2880,2710628,2722360,2600.876520,2585.000,4.572438e+04,0.006104,2710628,2722360,11732


In [15]:
trades = pd.DataFrame(stats['_trades'])

In [16]:
trades.to_sql(f"trades_{table}", connection, if_exists="replace")

In [17]:
trades.to_excel(f"trades_{table}.xlsx")

In [18]:
from risk_metrics import Risk_Metrics

In [19]:
Risk_Metrics(trades, df_long, 0).sharpe_ratio("momentum_1day")

201.72893688204582

In [20]:
Risk_Metrics(trades, df_long, 0).sortino_ratio("momentum_1day")

1344.5846765038534

In [21]:
Risk_Metrics(trades, df_long, 0).max_drawdown()

0.572594501784519

In [22]:
Risk_Metrics(trades, df_long, 0).calmar_ratio(stats["Return [%]"]) 

33590.79721836723